In [4]:
import cv2
import time
import mediapipe as mp

In [8]:
cap=cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)

mpHand =mp.solutions.hands
hands=mpHand.Hands()
mpDraw=mp.solutions.drawing_utils # elimizdeki isketleti çizmesi için

tipIds=[4,8,12,16,20] # Parmaklarımızın uçlarının id'leri

while True:
    
    success,img=cap.read()
    imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB) # mediapipe RGB kabul ediyor.
    results=hands.process(imgRGB)
    
    lmList=[]
    
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            mpDraw.draw_landmarks(img,handLms,mpHand.HAND_CONNECTIONS)
            
            for id, lm in enumerate(handLms.landmark):
                
                h,w,_=img.shape
                
                cx,cy =int(lm.x*w),int(lm.y*h) # Koordinatlar
                lmList.append([id,cx,cy])  # Id ve koordinatlar
                
    if len(lmList) !=0:
        fingers = []
        if lmList[tipIds[0]-2][1] > lmList[tipIds[4]-3][1]: #Sağ el
            # baş parmak
            if lmList[tipIds[0]][1] > lmList[tipIds[0]-1][1]:
                fingers.append(1)
            else:
                fingers.append(0)
        else:
            # baş parmak
            if lmList[tipIds[0]][1] < lmList[tipIds[0]-1][1]:
                fingers.append(1)
            else:
                fingers.append(0)
        # Diğer 4 parmak 
        
        for id in range (1,5):
            
            if lmList[tipIds[id]][2] < lmList[tipIds[id] - 2][2]:
                fingers.append(1)
            else: 
                fingers.append(0)
                
        total = fingers.count(1)
        
        cv2.putText(img, str(total),(30,125),cv2.FONT_HERSHEY_PLAIN,10,(255,0,0),8)
                
    cv2.imshow("img",img)
    if cv2.waitKey(1) & 0xFF == ord("q"): break
cap.release() #serbest bırak
cv2.destroyAllWindows()

  Bu algoritmamızda ilk olarak parmak uçlarımızın id'sini aldık.Daha sonra parmak ucumuzun id'sinin y koordinatı 2 altındaki eklemin id'sinin y koordinatından büyük mü diye baktık eğer öyleyse parmaklarımızı kapatmışız demektir.Bu kısma sadece baş parmak uymadı onuda bir altındaki eklemin sağındamı solunda mı olduğunu kontrol ederek çözdük.Eğer parmak kapalıysa 0 açık ise 1 olarak listeledik.Bu algoritma 2 el ile de çalışır.Bunu 17. id'ye sahip olan eklemin, 2 numaralı id'ye sahip olan ekleme göre x koordinatının büyük veya küçük olmasına göre belirledik.